Gaussian Mixture


In [105]:
import pandas as pd
import numpy as np

In [106]:
df = pd.read_csv('combined_data.csv')
df

,current_1,current_2,current_3
0,2040,2361,38
1,2031,2365,52
2,2031,2370,53
3,2025,2372,54
4,2023,2379,54
...,...,...,...
3169995,2214,1978,368
3169996,2217,1985,358
3169997,2218,1989,346
3169998,2220,1996,337


In [107]:
features = df.groupby(df.index // 10000).agg(['mean', 'var', 'std'])

In [108]:
features.head()

current_1                            current_2                            \
        mean           var         std       mean           var         std   
0  1987.6003  32783.557096  181.062302  2174.0711  33020.711316  181.716018   
1  1982.4803  33379.063118  182.699379  2165.3532  32815.538604  181.150596   
2  1980.2381  33015.768385  181.702417  2170.6435  32957.610569  181.542311   
3  1978.7542  33224.950477  182.277125  2170.5052  32863.028676  181.281628   
4  1982.3731  33032.491746  181.748430  2164.9059  33034.596305  181.754219   

  current_3                            
       mean           var         std  
0  440.3539  75256.979553  274.330056  
1  435.3327  73915.736984  271.874488  
2  428.8870  74261.647196  272.509903  
3  428.6128  74053.162992  272.127108  
4  432.9330  74385.515663  272.737082

In [109]:
from sklearn.mixture import GaussianMixture

In [110]:
gmm = GaussianMixture(n_components=1, covariance_type='full')

In [111]:
gmm.fit(features)

c:\Users\deepdesk\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


GaussianMixture()

In [112]:
features.sample(frac=0.5)

current_1                            current_2                            \
          mean           var         std       mean           var         std   
228  2003.9889  32651.725749  180.697885  2192.2902  32766.805465  181.016036   
239  2014.5465  32557.083946  180.435817  2201.4372  33086.332489  181.896488   
37   1976.7764  32833.822985  181.201057  2159.6518  32747.484505  180.962661   
148  1986.8928  33304.598568  182.495475  2175.2171  32620.161584  180.610525   
281  1988.3733  32987.069854  181.623429  2169.4157  32866.781772  181.291979   
..         ...           ...         ...        ...           ...         ...   
280  1988.1944  33130.757484  182.018564  2168.7306  32913.822606  181.421671   
286  1984.3693  32884.494567  181.340824  2172.2666  33412.711996  182.791444   
14   1980.8792  32985.888996  181.620178  2163.2668  33116.104028  181.978306   
47   1979.4071  33275.844354  182.416678  2164.0234  32693.256578  180.812767   
39   1976.8843  33219.256039  182.261505  2163.2896  33032.568589  181.748641   

    current_3                            
         mean           var         std  
228  467.9146  74652.066513  273.225304  
239  492.1805  74070.219942  272.158446  
37   422.5502  75082.748755  274.012315  
148  437.7059  72929.218527  270.054103  
281  439.0036  74521.662353  272.986561  
..        ...           ...         ...  
280  441.1613  74234.687951  272.460434  
286  441.2295  73979.491779  271.991713  
14   427.7127  74242.114970  272.474063  
47   430.0876  74002.668393  272.034315  
39   422.0684  74069.636285  272.157374  

[158 rows x 9 columns]

In [113]:
eval_df = features.sample(frac=0.5)

In [114]:
scores = gmm.score_samples(features)

In [115]:
scores.max(),scores.min(),scores.mean()

(-20.28707969507137, -177.99967498524245, -24.356203424104997)

In [116]:

anomaly_threshold = gmm.score_samples(eval_df) < -30


In [118]:
import copy
eval_df_1 = copy.deepcopy(eval_df)

Self evaluating

In [119]:
eval_df_1['anomaly'] = anomaly_threshold.astype(int)

In [120]:
len(eval_df_1.loc[eval_df_1['anomaly'] == 1])

8

In [122]:
eval_df_1.loc[eval_df_1['anomaly'] == 1]

current_1                            current_2                            \
          mean           var         std       mean           var         std   
265  1984.5318  32849.027892  181.243008  2165.6570  32557.458897  180.436856   
239  2014.5465  32557.083946  180.435817  2201.4372  33086.332489  181.896488   
148  1986.8928  33304.598568  182.495475  2175.2171  32620.161584  180.610525   
216  1957.9241  32307.773117  179.743632  2145.4252  32878.510656  181.324325   
182  1986.4803  32960.364448  181.549895  2171.4687  31709.078528  178.070431   
230  2015.5736  32472.498233  180.201271  2198.6288  32220.627273  179.501051   
235  2018.2980  32578.728269  180.495785  2194.4414  32565.410107  180.458888   
195  1970.0554  32621.095840  180.613111  2154.0853  33104.757700  181.947129   

    current_3                           anomaly  
         mean           var         std          
265  441.0816  75958.101352  275.604973       1  
239  492.1805  74070.219942  272.158446       1  
148  437.7059  72929.218527  270.054103       1  
216  395.5987  73677.876246  271.436689       1  
182  442.4450  74781.880563  273.462759       1  
230  493.8677  75457.086305  274.694533       1  
235  494.5467  75094.319051  274.033427       1  
195  412.6812  72494.528619  269.248080       1

possible real time function

In [ ]:
# while True:
#     current_readings = get_current_readings()
#     current_features = pd.DataFrame([current_readings]).agg(['mean', 'var', 'std'])
#     is_anomaly = gmm.score_samples(current_features) < -30
#     if is_anomaly:
#         alert_operator()

Adding Anomalies 

In [131]:
def add_anomalies(dataframe, number_of_anomalies=0, data_length=0):

        # Determine the upper bound of the data to which anomalies should be added.
        upper_bound = number_of_anomalies if number_of_anomalies else len(dataframe)
        
        # Determine the length of the data to which anomalies should be added.
        length = data_length if data_length else len(dataframe)

        # Make a deep copy of the data up to the upper bound and length.
        df = copy.deepcopy(dataframe[0:length])
        noise_df = df.iloc[:upper_bound,:]

        # Calculate the noise amplitude based on the signal amplitude.
        noise_amplitude = 0.1 * np.max(np.abs(df))

        # Add noise to the signal using amplitude scaling.
        noise = np.random.normal(0, noise_amplitude, noise_df.shape)
        
        df.iloc[:upper_bound,:] += noise

        return df


In [132]:
anomalous_data = add_anomalies(df,5000,10000)

c:\Users\deepdesk\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [126]:
anomalous_features = anomalous_data.groupby(anomalous_data.index // 10000).agg(['mean', 'var', 'std'])


In [127]:
anomalous_features

current_1                              current_2                \
          mean           var         std         mean           var   
0  1989.126637  58085.462209  241.009258  2172.866192  62663.309488   

                current_3                            
          std        mean           var         std  
0  250.326406  440.471106  79182.756301  281.394308

In [128]:
is_anomaly = gmm.score_samples(anomalous_features) < -30
if is_anomaly:
    print('anomalous')

anomalous


In [129]:
normal_features = df[:10000].groupby(df[:10000].index // 10000).agg(['mean', 'var', 'std'])


In [130]:
is_anomaly = gmm.score_samples(normal_features) < -30
if is_anomaly:
    print('anomalous')

In [ ]:
#TODO use the harmonics and other features here and verify